In [3]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [9]:
# Loding data
metadata = pd.read_csv('data/HAM10000_metadata.csv')

In [10]:
metadata.head(5)

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [6]:
# Loding images
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    image = load_img(image_path, target_size=target_size)
    image = img_to_array(image)
    image = image / 255.0
    return image

In [13]:
import tensorflow as tf


# Split the dataset
train_df, val_df = train_test_split(metadata, test_size=0.2, random_state=42)

# Image data generator with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=90
)
val_datagen = ImageDataGenerator(rescale=1./255)

# Directory of the images
image_dir = 'data/HAM10000_images'

# Create a function to load images from directory
def load_image(image_id):
    return os.path.join(image_dir, f"{image_id}.jpg")

# Add file path to the DataFrame
train_df['image_path'] = train_df['image_id'].apply(load_image)
val_df['image_path'] = val_df['image_id'].apply(load_image)

# Create data generators
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='image_path',
    y_col='dx',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32
)

val_generator = val_datagen.flow_from_dataframe(
    val_df,
    x_col='image_path',
    y_col='dx',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32
)

# Get the number of classes
num_classes = len(train_generator.class_indices)

Found 8012 validated image filenames belonging to 7 classes.
Found 2003 validated image filenames belonging to 7 classes.


# Model Training

In [15]:
from tensorflow.keras.applications import MobileNetV2


# Load the MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks
checkpoint = ModelCheckpoint('mobilenetv2_skin_cancer.keras', monitor='val_accuracy', save_best_only=True, mode='max')
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, mode='max')

# Train the model
history = model.fit(train_generator, epochs=20, validation_data=val_generator, callbacks=[checkpoint, early_stopping])

# Unfreeze some layers and fine-tune
for layer in base_model.layers[-50:]:
    layer.trainable = True

# Recompile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Continue training
history_fine = model.fit(train_generator, epochs=10, validation_data=val_generator, callbacks=[checkpoint, early_stopping])


9406464/9406464 [==============================] - 9s 1us/step
Epoch 1/20
251/251 [==============================] - 326s 1s/step - loss: 0.9082 - accuracy: 0.7032 - val_loss: 0.8361 - val_accuracy: 0.7134
Epoch 2/20
251/251 [==============================] - 289s 1s/step - loss: 0.7267 - accuracy: 0.7420 - val_loss: 0.7450 - val_accuracy: 0.7374
Epoch 3/20
251/251 [==============================] - 299s 1s/step - loss: 0.6770 - accuracy: 0.7552 - val_loss: 0.7517 - val_accuracy: 0.7394
Epoch 4/20
251/251 [==============================] - 282s 1s/step - loss: 0.6591 - accuracy: 0.7596 - val_loss: 0.7612 - val_accuracy: 0.7169
Epoch 5/20
251/251 [==============================] - 292s 1s/step - loss: 0.6296 - accuracy: 0.7742 - val_loss: 0.7350 - val_accuracy: 0.7299
Epoch 6/20
251/251 [==============================] - 287s 1s/step - loss: 0.6084 - accuracy: 0.7807 - val_loss: 0.7197 - val_accuracy: 0.7404
Epoch 7/20
251/251 [==============================] - 286s 1s/step - loss: 0.60

# Model Evaluation

In [16]:
# Load the best model
model.load_weights('mobilenetv2_skin_cancer.keras')

# Evaluate on validation set
val_loss, val_accuracy = model.evaluate(val_generator)
print(f'Validation Accuracy: {val_accuracy * 100:.2f}%')

63/63 [==============================] - 36s 568ms/step - loss: 0.6141 - accuracy: 0.7993
Validation Accuracy: 79.93%


# Web app

In [27]:
from keras.models import load_model

# Assuming you have a loaded model
model = load_model('mobilenetv2_skin_cancer.keras')

# Save the model to the correct path
model.save('C:/Users/HP\Desktop/ai health web/mobilenetv2_skin_cancer.h5')


# web try 

# Web try 2

In [1]:
from flask import Flask, request, jsonify, render_template_string
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import threading
from io import BytesIO

# Define the Flask app
app = Flask(__name__)

# Load your pre-trained model
model = load_model('mobilenetv2_skin_cancer.keras')

# Assuming you have a mapping of class indices to labels and recommendations
labels = {
    0: 'akiec', 
    1: 'bcc', 
    2: 'bkl', 
    3: 'df', 
    4: 'mel', 
    5: 'nv', 
    6: 'vasc'
}

recommendations = {
    'akiec': "Actinic keratosis: Precancerous condition; consult a dermatologist for treatment options.",
    'bcc': "Basal cell carcinoma: Seek medical advice for treatment options.",
    'bkl': "Benign keratosis: Usually harmless but monitor for changes. Consult a dermatologist if needed.",
    'df': "Dermatofibroma: Generally benign; consult a dermatologist if there are concerns.",
    'mel': "Melanoma: Consult a dermatologist immediately for further evaluation and treatment.",
    'nv': "Nevus (mole): Generally benign, but keep an eye on changes in size, shape, or color.",
    'vasc': "Vascular lesion: Generally benign; consult a dermatologist if there are concerns."
}

@app.route('/')
def home():
    return render_template_string('''
    <!DOCTYPE html>
    <html>
    <head>
        <title>Skin Lesion Detection</title>
    </head>
    <body>
        <h1>Upload a skin lesion image for detection</h1>
        <form id="predictionForm" enctype="multipart/form-data">
            <input type="file" id="inputData" name="file">
            <button type="button" onclick="makePrediction()">Predict</button>
        </form>
        <h2>Prediction:</h2>
        <div id="result"></div>

        <script>
            function makePrediction() {
                const formData = new FormData();
                formData.append('file', document.getElementById('inputData').files[0]);
                
                fetch('/predict', {
                    method: 'POST',
                    body: formData
                })
                .then(response => response.text())  // Changed to text() to receive HTML response
                .then(data => {
                    document.getElementById('result').innerHTML = data;  // Update innerHTML with response
                })
                .catch(error => {
                    console.error('Error:', error);
                });
            }
        </script>
    </body>
    </html>
    ''')

@app.route('/predict', methods=['POST'])
def predict():
    # Get the file from the POST request
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'}), 400
    
    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400
    
    try:
        # Load and preprocess the image
        img = image.load_img(BytesIO(file.read()), target_size=(224, 224))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array /= 255.0

        # Make predictions
        predictions = model.predict(img_array)
        predicted_class = np.argmax(predictions, axis=1)

        # Ensure predicted_class is in the range of labels keys
        if predicted_class[0] not in labels:
            return jsonify({'error': f'Invalid prediction class index {predicted_class[0]}'}), 500
        
        label = labels[predicted_class[0]]
        recommendation = recommendations.get(label, 'No recommendation found')

        # Render HTML response with prediction and recommendation
        return render_template_string('''
            <h3>Prediction: {{ prediction }}</h3>
            <p>Recommendation: {{ recommendation }}</p>
        ''', prediction=label, recommendation=recommendation)

    except Exception as e:
        return jsonify({'error': str(e)}), 500

def run_app():
    app.run(port=5000)

# Run the Flask app in a separate thread
flask_thread = threading.Thread(target=run_app)
flask_thread.start()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [29/Jun/2024 13:50:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2024 13:50:21] "GET / HTTP/1.1" 200 -


1/1 [==============================] - 41s 41s/step


127.0.0.1 - - [29/Jun/2024 13:51:15] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2024 13:51:15] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2024 13:51:15] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2024 13:51:15] "POST /predict HTTP/1.1" 200 -


1/1 [==============================] - 0s 84ms/step


127.0.0.1 - - [29/Jun/2024 13:55:22] "POST /predict HTTP/1.1" 200 -


In [2]:
!pip list > installed_libraries.txt
